# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 27 2022 3:15PM,242897,20,8325406,"Exact-Rx, Inc.",Released
1,Jul 27 2022 3:15PM,242897,20,8325407,"Exact-Rx, Inc.",Released
2,Jul 27 2022 3:15PM,242897,20,8325408,"Exact-Rx, Inc.",Released
3,Jul 27 2022 3:15PM,242897,20,8325409,"Exact-Rx, Inc.",Released
4,Jul 27 2022 3:15PM,242897,20,8325410,"Exact-Rx, Inc.",Released
5,Jul 27 2022 3:15PM,242897,20,8325411,"Exact-Rx, Inc.",Released
6,Jul 27 2022 3:15PM,242897,20,8325412,"Exact-Rx, Inc.",Released
7,Jul 27 2022 3:15PM,242897,20,8325413,"Exact-Rx, Inc.",Released
8,Jul 27 2022 3:15PM,242897,20,8325414,"Exact-Rx, Inc.",Released
9,Jul 27 2022 3:15PM,242896,10,8331073,"ClearSpec, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,242892,Released,2
38,242893,Released,1
39,242895,Released,1
40,242896,Released,1
41,242897,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242892,NaN,NaN,2.0
242893,NaN,NaN,1.0
242895,NaN,NaN,1.0
242896,NaN,NaN,1.0
242897,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242798,0.0,0.0,2.0
242808,17.0,3.0,4.0
242810,0.0,0.0,8.0
242818,14.0,6.0,1.0
242819,4.0,1.0,23.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242798,0,0,2
242808,17,3,4
242810,0,0,8
242818,14,6,1
242819,4,1,23


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242798,0,0,2
1,242808,17,3,4
2,242810,0,0,8
3,242818,14,6,1
4,242819,4,1,23


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242798,,,2
1,242808,17,3,4
2,242810,,,8
3,242818,14,6,1
4,242819,4,1,23


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 27 2022 3:15PM,242897,20,"Exact-Rx, Inc."
9,Jul 27 2022 3:15PM,242896,10,"ClearSpec, LLC"
10,Jul 27 2022 3:12PM,242895,10,Emerginnova
11,Jul 27 2022 3:09PM,242889,20,"ACI Healthcare USA, Inc."
14,Jul 27 2022 2:56PM,242893,19,"NAPP Technologies, LLC"
15,Jul 27 2022 2:54PM,242892,10,Emerginnova
17,Jul 27 2022 2:44PM,242891,10,Bio-PRF
18,Jul 27 2022 2:40PM,242886,20,Alumier Labs Inc.
38,Jul 27 2022 2:16PM,242887,12,LF of America Corp.
39,Jul 27 2022 1:46PM,242877,16,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 27 2022 3:15PM,242897,20,"Exact-Rx, Inc.",,,9
1,Jul 27 2022 3:15PM,242896,10,"ClearSpec, LLC",,,1
2,Jul 27 2022 3:12PM,242895,10,Emerginnova,,,1
3,Jul 27 2022 3:09PM,242889,20,"ACI Healthcare USA, Inc.",,,3
4,Jul 27 2022 2:56PM,242893,19,"NAPP Technologies, LLC",,,1
5,Jul 27 2022 2:54PM,242892,10,Emerginnova,,,2
6,Jul 27 2022 2:44PM,242891,10,Bio-PRF,,,1
7,Jul 27 2022 2:40PM,242886,20,Alumier Labs Inc.,,,20
8,Jul 27 2022 2:16PM,242887,12,LF of America Corp.,,,1
9,Jul 27 2022 1:46PM,242877,16,Sartorius Stedim North America,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 3:15PM,242897,20,"Exact-Rx, Inc.",9,,
1,Jul 27 2022 3:15PM,242896,10,"ClearSpec, LLC",1,,
2,Jul 27 2022 3:12PM,242895,10,Emerginnova,1,,
3,Jul 27 2022 3:09PM,242889,20,"ACI Healthcare USA, Inc.",3,,
4,Jul 27 2022 2:56PM,242893,19,"NAPP Technologies, LLC",1,,
5,Jul 27 2022 2:54PM,242892,10,Emerginnova,2,,
6,Jul 27 2022 2:44PM,242891,10,Bio-PRF,1,,
7,Jul 27 2022 2:40PM,242886,20,Alumier Labs Inc.,20,,
8,Jul 27 2022 2:16PM,242887,12,LF of America Corp.,1,,
9,Jul 27 2022 1:46PM,242877,16,Sartorius Stedim North America,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 3:15PM,242897,20,"Exact-Rx, Inc.",9,,
1,Jul 27 2022 3:15PM,242896,10,"ClearSpec, LLC",1,,
2,Jul 27 2022 3:12PM,242895,10,Emerginnova,1,,
3,Jul 27 2022 3:09PM,242889,20,"ACI Healthcare USA, Inc.",3,,
4,Jul 27 2022 2:56PM,242893,19,"NAPP Technologies, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 3:15PM,242897,20,"Exact-Rx, Inc.",9.0,NaN,NaN
1,Jul 27 2022 3:15PM,242896,10,"ClearSpec, LLC",1.0,NaN,NaN
2,Jul 27 2022 3:12PM,242895,10,Emerginnova,1.0,NaN,NaN
3,Jul 27 2022 3:09PM,242889,20,"ACI Healthcare USA, Inc.",3.0,NaN,NaN
4,Jul 27 2022 2:56PM,242893,19,"NAPP Technologies, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 3:15PM,242897,20,"Exact-Rx, Inc.",9.0,0.0,0.0
1,Jul 27 2022 3:15PM,242896,10,"ClearSpec, LLC",1.0,0.0,0.0
2,Jul 27 2022 3:12PM,242895,10,Emerginnova,1.0,0.0,0.0
3,Jul 27 2022 3:09PM,242889,20,"ACI Healthcare USA, Inc.",3.0,0.0,0.0
4,Jul 27 2022 2:56PM,242893,19,"NAPP Technologies, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3400044,41.0,32.0,1.0
12,242887,1.0,0.0,0.0
15,242808,4.0,3.0,17.0
16,242877,0.0,1.0,0.0
19,971340,5.0,6.0,14.0
20,1214353,63.0,1.0,4.0
21,971426,4.0,0.0,0.0
22,485703,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3400044,41.0,32.0,1.0
1,12,242887,1.0,0.0,0.0
2,15,242808,4.0,3.0,17.0
3,16,242877,0.0,1.0,0.0
4,19,971340,5.0,6.0,14.0
5,20,1214353,63.0,1.0,4.0
6,21,971426,4.0,0.0,0.0
7,22,485703,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,41.0,32.0,1.0
1,12,1.0,0.0,0.0
2,15,4.0,3.0,17.0
3,16,0.0,1.0,0.0
4,19,5.0,6.0,14.0
5,20,63.0,1.0,4.0
6,21,4.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,41.0
1,12,Released,1.0
2,15,Released,4.0
3,16,Released,0.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,17.0,0.0,14.0,4.0,0.0,0.0
Executing,32.0,0.0,3.0,1.0,6.0,1.0,0.0,0.0
Released,41.0,1.0,4.0,0.0,5.0,63.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,17.0,0.0,14.0,4.0,0.0,0.0
1,Executing,32.0,0.0,3.0,1.0,6.0,1.0,0.0,0.0
2,Released,41.0,1.0,4.0,0.0,5.0,63.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,17.0,0.0,14.0,4.0,0.0,0.0
1,Executing,32.0,0.0,3.0,1.0,6.0,1.0,0.0,0.0
2,Released,41.0,1.0,4.0,0.0,5.0,63.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()